In [1]:
!curl -s https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py > module_update.py

In [2]:
from module_update import update, css
update('nbpictures')
import json
import requests
import nbpictures as nbp
import nbtext as nb
from PIL import Image
from IPython.display import HTML
%matplotlib inline

Updated file `C:\Users\larsj\Documents\GitHub\illustrations_bokhylla\nbpictures.py`

In [13]:
nb.book_urn(author="%zwilgm%", title="%vi%børn%")

[[2016051948119, 'Zwilgmeyer, Dikken', 'Vi børn', 1900],
 [2009060913005, 'Zwilgmeyer, Dikken', 'Vi børn', 1890],
 [2016051948114, 'Zwilgmeyer, Dikken', 'Vi børn', 1893]]

In [3]:
nbp.iiif_manifest("URN:NBN:no-nb_digibok_2016100306030")

{'@context': 'http://iiif.io/api/presentation/2/context.json',
 '@type': 'sc:Manifest',
 '@id': 'https://api.nb.no/catalog/v1/iiif/URN:NBN:no-nb_digibok_2016100306030/manifest{?fields,profile}',
 'label': 'Villmarksvers',
 'metadata': [{'label': 'Tilgang', 'value': 'Tilgang for norske IP-adresser'},
  {'label': 'Tittel', 'value': 'Villmarksvers'},
  {'label': 'Forfatter', 'value': 'Skogen, Olav'},
  {'label': 'Illustratør', 'value': 'Enger, Erling'},
  {'label': 'Publisert', 'value': 'Oslo:Dreyer, 1946'},
  {'label': 'Andre opplysninger',
   'value': 'Elektronisk reproduksjon [Norge] Nasjonalbiblioteket Digital 2016-11-03\n'},
  {'label': 'Språk', 'value': 'Norsk (Bokmål)'},
  {'label': 'Kilde for metadata',
   'value': 'nb.bibsys.no (<a title="Link til post i Oria" href="https://www.oria.no/?vid=NB&search=990004716234702202">990004716234702202</a>)'},
  {'label': 'Omfang', 'value': '58 s. ill.'},
  {'label': 'Medietype', 'value': 'bøker'},
  {'label': 'UDC', 'value': '839.6'},
  {'lab

## Kode for å finne bilder og manipulere dem

In [53]:
small_scale = 0.59
large_scale = 1.58

def iiif_manifest(urn):
    r = requests.get("https://api.nb.no/catalog/v1/iiif/{urn}/manifest".format(urn=urn))
    return r.json()


    
def load_picture(url):
    r = requests.get(url, stream=True)
    r.raw.decode_content=True
    #print(r.status_code)
    return r.raw

def metadata(ill):
    #print(ill)
    u = bookurn_from_illustration(get_url_from_illustration(ill, part=100))
    m = iiif_manifest(u)['metadata']
    return m

def meta2html(ill):
    d = metadata(ill)
    ls = ""
    for i in d:
        ls += "<dt>{label}</dt><dd>{value}</dd>".format(label=i['label'], value=i['value'])
    return "<dl>{ls}</dl>".format(ls=ls)

def bookurn_from_illustration(ill):
    import re
    #print(ill)
    hits = re.findall("URN[^/]+", ill)
    if hits != []:
        res = hits[0].split('_')[:-1]
        res = '_'.join(res)
    return res

def get_url_from_illustration(illus, part = 200, scale = small_scale):
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(int(illus['width'])*scale), 
            height = int(int(illus['height'])*scale), 
            vpos = int(int(illus['vpos'])*scale), 
            hpos = int(int(illus['hpos'])*scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_url(illus, part = 200):
    """part sets size of output of page, if part is True it returns the cut out of image
    illus is a dictionary of with entries and values like this: 
    {'height': 270, 'hpos': 251, 'page': 'digibok_2017081626006_0018', 'resolution': 400, 'vpos': 791, 'width': 373} """
    
    if illus['resolution'] >= 400:
        scale = large_scale
    else:
        scale = small_scale
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(int(illus['width'])*scale), 
            height = int(int(illus['height'])*scale), 
            vpos = int(int(illus['vpos'])*scale), 
            hpos = int(int(illus['hpos'])*scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_picture_from_illustration(illus, page=True, scale=small_scale):
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def get_picture(illus, page=True, scale = small_scale):
    if int(illus['resolution']) >= 400:
        scale = large_scale
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def display_finds(r):
    """A list of urls in r is displayed as HTML"""
    rows = ["<tr><td><img src='{row}'</td><td>{meta}</td></tr>".format(row=row, meta=row) for row in r]
    return HTML("""<html><head></head>
     <body>
     <table>
     {rows}
     </table>
     </body>
     </html>
     """.format(rows=' '.join(rows)))



In [61]:
ills = [{'height': 424,
  'hpos': 1203,
  'page': 'digibok_2016090708313_0029',
  'resolution': 400,
  'vpos': 1715,
  'width': 183},
 {'height': 436,
  'hpos': 203,
  'page': 'digibok_2016090708313_0031',
  'resolution': 400,
  'vpos': 1850,
  'width': 863},
 {'height': 349,
  'hpos': 206,
  'page': 'digibok_2016090708313_0035',
  'resolution': 400,
  'vpos': 1289,
  'width': 1316},
 {'height': 312,
  'hpos': 244,
  'page': 'digibok_2016090708313_0037',
  'resolution': 400,
  'vpos': 427,
  'width': 1115},
 {'height': 244,
  'hpos': 234,
  'page': 'digibok_2016090708313_0037',
  'resolution': 400,
  'vpos': 2005,
  'width': 1310},
 {'height': 242,
  'hpos': 217,
  'page': 'digibok_2016090708313_0039',
  'resolution': 400,
  'vpos': 535,
  'width': 1317},
 {'height': 231,
  'hpos': 358,
  'page': 'digibok_2016090708313_0040',
  'resolution': 400,
  'vpos': 556,
  'width': 1310},
 {'height': 240,
  'hpos': 231,
  'page': 'digibok_2016090708313_0041',
  'resolution': 400,
  'vpos': 530,
  'width': 1317},
 {'height': 684,
  'hpos': 221,
  'page': 'digibok_2016090708313_0043',
  'resolution': 400,
  'vpos': 1488,
  'width': 1318},
 {'height': 687,
  'hpos': 208,
  'page': 'digibok_2016090708313_0045',
  'resolution': 400,
  'vpos': 1085,
  'width': 1369},
 {'height': 638,
  'hpos': 364,
  'page': 'digibok_2016090708313_0047',
  'resolution': 400,
  'vpos': 1347,
  'width': 838},
 {'height': 543,
  'hpos': 1182,
  'page': 'digibok_2016090708313_0049',
  'resolution': 400,
  'vpos': 1504,
  'width': 220},
 {'height': 482,
  'hpos': 1326,
  'page': 'digibok_2016090708313_0050',
  'resolution': 400,
  'vpos': 787,
  'width': 204},
 {'height': 908,
  'hpos': 830,
  'page': 'digibok_2016090708313_0067',
  'resolution': 400,
  'vpos': 1812,
  'width': 940},
 {'height': 177,
  'hpos': 419,
  'page': 'digibok_2016090708313_0069',
  'resolution': 400,
  'vpos': 869,
  'width': 1247},
 {'height': 2110,
  'hpos': 976,
  'page': 'digibok_2016090708313_0071',
  'resolution': 400,
  'vpos': 608,
  'width': 833},
 {'height': 1880,
  'hpos': 769,
  'page': 'digibok_2016090708313_0073',
  'resolution': 400,
  'vpos': 419,
  'width': 928},
 {'height': 665,
  'hpos': 1319,
  'page': 'digibok_2016090708313_0074',
  'resolution': 400,
  'vpos': 1410,
  'width': 407},
 {'height': 1689,
  'hpos': 27,
  'page': 'digibok_2016090708313_0079',
  'resolution': 400,
  'vpos': 1041,
  'width': 1737},
 {'height': 1570,
  'hpos': 1245,
  'page': 'digibok_2016090708313_0080',
  'resolution': 400,
  'vpos': 371,
  'width': 229},
 {'height': 319,
  'hpos': 1393,
  'page': 'digibok_2016090708313_0081',
  'resolution': 400,
  'vpos': 446,
  'width': 342},
 {'height': 302,
  'hpos': 772,
  'page': 'digibok_2016090708313_0081',
  'resolution': 400,
  'vpos': 1226,
  'width': 475},
 {'height': 576,
  'hpos': 382,
  'page': 'digibok_2016090708313_0081',
  'resolution': 400,
  'vpos': 1819,
  'width': 1313},
 {'height': 1882,
  'hpos': 773,
  'page': 'digibok_2016090708313_0083',
  'resolution': 400,
  'vpos': 421,
  'width': 931},
 {'height': 881,
  'hpos': 907,
  'page': 'digibok_2016090708313_0084',
  'resolution': 400,
  'vpos': 623,
  'width': 687},
 {'height': 210,
  'hpos': 320,
  'page': 'digibok_2016090708313_0084',
  'resolution': 400,
  'vpos': 1548,
  'width': 1268},
 {'height': 572,
  'hpos': 224,
  'page': 'digibok_2016090708313_0027',
  'resolution': 400,
  'vpos': 1556,
  'width': 1317}]

In [76]:
urls = [get_url(ill, part = True) for ill in ills]

In [77]:
urls

['https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0029/1900,2709,289,669/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0031/320,2923,1363,688/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0035/325,2036,2079,551/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/385,674,1761,492/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/369,3167,2069,385/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0039/342,845,2080,382/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0040/565,878,2069,364/full/0/native.jpg',
 'https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0041/364,837,2080,379/full/0/native.jpg',
 'https://www.nb.no/

In [78]:
display_finds(urls)

"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0029/1900,2709,289,669/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0029/1900,2709,289,669/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0031/320,2923,1363,688/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0031/320,2923,1363,688/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0035/325,2036,2079,551/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0035/325,2036,2079,551/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/385,674,1761,492/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/385,674,1761,492/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/369,3167,2069,385/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0037/369,3167,2069,385/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0039/342,845,2080,382/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0039/342,845,2080,382/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0040/565,878,2069,364/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0040/565,878,2069,364/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0041/364,837,2080,379/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0041/364,837,2080,379/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0043/349,2351,2082,1080/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0043/349,2351,2082,1080/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0045/328,1714,2163,1085/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0045/328,1714,2163,1085/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0047/575,2128,1324,1008/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2016090708313_0047/575,2128,1324,1008/full/0/native.jpg"


In [56]:
iiif_manifest("URN:NBN:no-nb_digibok_2016100306030")

{'@context': 'http://iiif.io/api/presentation/2/context.json',
 '@type': 'sc:Manifest',
 '@id': 'https://api.nb.no/catalog/v1/iiif/URN:NBN:no-nb_digibok_2016100306030/manifest{?fields,profile}',
 'label': 'Villmarksvers',
 'metadata': [{'label': 'Tilgang', 'value': 'Tilgang for norske IP-adresser'},
  {'label': 'Tittel', 'value': 'Villmarksvers'},
  {'label': 'Forfatter', 'value': 'Skogen, Olav'},
  {'label': 'Illustratør', 'value': 'Enger, Erling'},
  {'label': 'Publisert', 'value': 'Oslo:Dreyer, 1946'},
  {'label': 'Andre opplysninger',
   'value': 'Elektronisk reproduksjon [Norge] Nasjonalbiblioteket Digital 2016-11-03\n'},
  {'label': 'Språk', 'value': 'Norsk (Bokmål)'},
  {'label': 'Kilde for metadata',
   'value': 'nb.bibsys.no (<a title="Link til post i Oria" href="https://www.oria.no/?vid=NB&search=990004716234702202">990004716234702202</a>)'},
  {'label': 'Omfang', 'value': '58 s. ill.'},
  {'label': 'Medietype', 'value': 'bøker'},
  {'label': 'UDC', 'value': '839.6'},
  {'lab